In [ ]:
import pandas as pd
import numpy as np 
import json 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import itertools
import re
import plotly
import plotly.graph_objects as go
import plotly.express as px
from itertools import chain
import ast
from IPython.display import HTML
import spacy
from spacy import displacy
from spacy.tokens import Doc, Span
spacy_nlp = spacy.load('en_core_web_sm')
from spacy.lang.en import English
import string
en_tokenizer = English().tokenizer


## Load train data

In [ ]:
# Load data in dictionary and dataframe format
with open('../data/train.json', 'r') as f:
    data = json.load(f)

data_df = pd.read_json('../data/train.json')

In [ ]:
data[0].keys()

In [ ]:
data_df.head(2)

In [ ]:
print('Number of documents:',len(data))

In [ ]:
#preprocess data
data_df = data_df.rename(columns={'full_text':'text'})
data_df['llm_generated'] = False
data_df['prompt_id'] = 0

## Load llm generated data 

In [ ]:
# Load more data (llm generated)
pii_dataset = pd.read_csv('../data/pii_dataset.csv')

In [ ]:
pii_dataset.head(2)

In [ ]:
print('Number of documents:',len(pii_dataset))

In [ ]:
# preprocess data
pii_dataset['llm'] = True

# Convert string to list
pii_dataset[["tokens", "trailing_whitespace", "labels"]] = pii_dataset[["tokens", "trailing_whitespace", "labels"]].map(ast.literal_eval)
pii_dataset["document"] = pii_dataset["document"].astype("category").cat.codes + (data_df.document.max() + 1) 


In [ ]:
def tokenize_with_spacy(text, tokenizer=en_tokenizer):
    tokenized_text = tokenizer(text)
    tokens = [token.text for token in tokenized_text]
    trailing_whitespace = [bool(token.whitespace_) for token in tokenized_text]
    return {'tokens': tokens, 'trailing_whitespace': trailing_whitespace}

def create_new_tokens_labels(row):
    '''
    We will convert tokens to spacy tokens. 
    This way we can have multiple spacy tokens with different labels seperated by no trailing whitespace. 
    If original tokens have always spaces, then tokenizing them individually to spacy tokens and then combining will be exactly 
    same sa tokenizing the whole text created by the original tokens.
    The labels are created such that if spacy token is punctuation, then it has 'O' label for time being.
    Also handle common mislabeled STREET-ADDRESS.
    '''
    
    tokens, labels, trailing_whitespace = row.tokens, row.labels, row.trailing_whitespace
    new_tokens, new_labels, new_trailing_whitespaces = [], [], []
    labels = [l.split("-")[1] if l != "O" else l for l in labels]
    
    for i in range(len(tokens)):
        t = tokens[i]
        l = labels[i]
        ws = trailing_whitespace[i]
        
        prev_l = labels[i - 1] if i > 0 else "O"
        next_l = labels[i + 1] if (i + 1) < len(labels) else "O"
        
        # Found a PHONE_NUM token mislabed as STREET_ADDRESS so make sure:
        if l != "O" and re.search(r'\+\d+', t):
            l = "PHONE_NUM"
        
        # Found STREET_ADDRESS between 2 PHONE_NUM so make sure:
        if l == "STREET_ADDRESS" and prev_l == "PHONE_NUM" and prev_l == next_l:
            l = "PHONE_NUM"
            
        # Found individual mislabeled STREET_ADDRESS tokens so make sure:
        elif l == "STREET_ADDRESS" and l != next_l and l != prev_l:
            l = "O"
        
        # Create spacy tokens and their labels
        tok_ = tokenize_with_spacy(t)
        spacy_tokens = tok_["tokens"]
        new_tokens.extend(spacy_tokens)
        
        new_labels.extend([l if st not in string.punctuation else "O" for st in spacy_tokens])
        
        new_trailing_whitespaces.extend(tok_["trailing_whitespace"])
        new_trailing_whitespaces[-1] = ws  
        
    return pd.Series({"document": row.document, "tokens": new_tokens, "trailing_whitespace": new_trailing_whitespaces, "labels": new_labels})


def update_labels(row):
    '''
    Update the labels of puntuation
    '''
    
    tokens = row.tokens
    labels = row.labels
    new_labels = ["O"] * len(labels)
    for i, (t, l) in enumerate(zip(tokens, labels)):
        prev_l = new_labels[i - 1] if i > 0 else "O"
        next_l = labels[i + 1] if i + 1 < len(labels) else "O"
    
        if (prev_l == "NAME_STUDENT" or prev_l == "O") and t == "'s":
            new_labels[i] = "O"

        elif t == "(" and next_l == "PHONE_NUM":
            new_labels[i] = next_l

        elif t == ")" and prev_l == "PHONE_NUM":
            new_labels[i] = prev_l
        
        # The sandwitched punctuaion between two same labels is given same label except some cases:
        elif (t in string.punctuation) and (prev_l == next_l) and (prev_l != "O"):
            
            if t == "," and prev_l != "STREET_ADDRESS":
                new_labels[i] = "O"
            elif t == "." and prev_l == "NAME_STUDENT":
                new_labels[i] = "O"
            else:
                new_labels[i] = prev_l
        
        else:
            new_labels[i] = l
    
    return new_labels

def create_bio_labels(labels):
    new_labels = ["O"]*len(labels)
    prev_l = "O"
    for i, l in enumerate(labels):
        if l != "O":
            if l != prev_l:
                new_labels[i] = "B-" + l
            elif l == prev_l:
                new_labels[i] = "I-" + l
        prev_l = l
    return new_labels

In [ ]:
new_pii_dataset = pii_dataset.apply(create_new_tokens_labels, axis=1)
new_pii_dataset["labels"] = new_pii_dataset.apply(update_labels, axis=1).apply(create_bio_labels)


In [ ]:
new_pii_dataset

In [ ]:
new_pii_dataset['text'] = new_pii_dataset['tokens'].apply(lambda x: ' '.join(x))



In [ ]:
# combine the two datasets
df = pd.concat([data_df, pii_dataset[['document', 'text', 'tokens','trailing_whitespace', 'labels', 'prompt_id', 'llm']]], ignore_index=True)

# EDA

## Target distribution plot

In [ ]:
labels = df['labels'].tolist()

# Flatten the list of lists
flattened_labels = list(itertools.chain.from_iterable(labels))

# Count the occurrences of each label
label_counts = Counter(flattened_labels)

# Separate the labels and their counts for plotting
labels, counts = zip(*label_counts.items())

In [ ]:
# Create the bar plot
fig = go.Figure([go.Bar(x=labels[1:], y=counts[1:])])

# Customize the layout
fig.update_layout(title_text='Frequency of Each Label', xaxis_title='Labels', yaxis_title='Frequency')

# Show the plot
fig.show()

## Unique targets in each document

In [ ]:
df['unique_labels'] = df['labels'].apply(lambda x: list(set(x)))
df['num_labels'] = df['unique_labels'].apply(len)

In [ ]:
#histogram of number of labels per document
fig = px.histogram(df, x='num_labels', nbins=20, title='Histogram of number of unique labels per document')

# Show the plot
fig.show()

## Distribution of tokens in each document (with whitespace) 

In [ ]:
df['num_tokens'] = df['tokens'].apply(len)

In [ ]:
#histogram of number of tokens per document
fig = px.histogram(df, x='num_tokens', nbins=500, title='Histogram of number of tokens per document')

# Show the plot
fig.show()

## Distribution of text length

In [ ]:
df['len_text'] = df['text'].apply(len)
fig = px.histogram(df, x='len_text', nbins=500, title='Histogram of length of text per document')

# Show the plot
fig.show()

## Distribution of documents without labels(only "o")

In [ ]:
df_non_outer = df[df['labels'].apply(lambda x: len(set(x)) > 1)] #with labels
df_outer = df[df['labels'].apply(lambda x: 'O' in x and len(set(x)) == 1)] #without labels 

In [ ]:
df_non_outer['Label Type'] = 'With Labels'
df_outer['Label Type'] = 'Without Labels'

# Calculate text length
df['len_text'] = df['text'].apply(len)
df_non_outer['len_text'] = df_non_outer['text'].apply(len)
df_outer['len_text'] = df_outer['text'].apply(len)

# Combine the dataframes
combined_df = pd.concat([df_non_outer, df_outer])

# Plotting
fig = px.histogram(combined_df, x='len_text', color='Label Type', barmode='overlay',
                   nbins=500, title='Histogram of Length of Text per Document')

# Show the plot
fig.show()

In [ ]:
# Combine the dataframes
combined_df = pd.concat([df_non_outer, df_outer])
combined_df['len_tokens'] = combined_df['tokens'].apply(len)
# Plotting
fig = px.histogram(combined_df, x='len_tokens', color='Label Type', barmode='overlay',
                   nbins=500, title='Histogram of Length of Tokens per Document')

# Show the plot
fig.show()

## Labels Pos

In [ ]:
df["labels_pos"] = df["labels"].apply(lambda labels: np.arange(1, len(labels) + 1) / len(labels))
exp_df = df.explode(["tokens", "labels", "labels_pos"])
exp_df["labels"] = pd.Categorical(exp_df["labels"], categories=labels, ordered=True)
exp_df = exp_df.sort_values(by="labels", ascending=False)
label_tokens = exp_df.groupby("labels", observed=False).agg(list)
label_tokens["counts"] = label_tokens["tokens"].apply(len)

In [ ]:
fig = px.scatter(exp_df, x='labels_pos', y='labels', title='Scatter Plot of Labels in Documents',)

fig.update_layout(
    xaxis_title='X Axis Label',
    yaxis_title='Y Axis Label',
    legend_title='Legend'
)
fig.show()

In [ ]:
word cloud for each target 
wordcloud for surronding words of each target


In [ ]:
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(df_train['full_text']))
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Essays')
plt.show()

In [ ]:
def bio_to_spacy_entities(doc, labels):
    """Convert BIO-tagged labels into spaCy entity spans.

    Args:
        doc (Doc): A spaCy Doc object containing the tokenized text.
        labels (List[str]): A list of BIO labels corresponding to each token in the Doc.

    Returns:
        List[Span]: A list of spaCy Span objects representing the entities.
    """
    entities = []
    start = None  # Track start index of the entity
    for i, (token, label) in enumerate(zip(doc, labels)):
        if label.startswith('B-'):
            start = i
        elif label.startswith('I-') and start is not None:
            continue
        elif start is not None:
            # End of the entity
            entities.append(Span(doc, start, i, label=labels[start][2:]))
            start = None
        if label.startswith('B-') and i == len(doc) - 1:
            # Handle case where an entity is at the end of the text
            entities.append(Span(doc, start, i+1, label=label[2:]))
    if start is not None:
        # Ensure any final entity is added
        entities.append(Span(doc, start, len(doc), label=labels[start][2:]))
    return entities

In [ ]:
def visualize_example(document_series, options=None, custom_css=None):
    """Visualize entities in a text using spaCy and displaCy.

    Args:
        document_series (dict | pd.Series): The example to visualize
        options (dict, optional): The options for displacy rendering colours
        custom_css (dict, optional): Additional customization
    """

    doc = spacy_nlp(document_series['text'])  # Tokenize the text with spaCy
    entities = bio_to_spacy_entities(doc, document_series['labels'])
    doc.ents = entities  # Update the doc's entities
    
    # Define custom CSS
    custom_css = custom_css or """
    <style>    
        /* Customizing entity appearance */
        .entities {
            font-size: 11px !important;
            font-family: Verdana !important;
            line-height: 1.25 !important;
            border-radius: 10px !important; /* Rounded corners */
            background-color: #f9f9f9 !important; /* Very light gray background */
            padding: 20px 15px !important; /* Adjust padding */
        }
        /* Customizing entity appearance */
        .entity {
            font-size: 10px !important;
            padding: 0.2em 0.4em !important;
            font-family: Verdana !important;
            font-weight: bold !important;
            
        }
    </style>
    """

    # Inject custom CSS
    display(HTML(custom_css))
    
    # Visualization
    displacy.render(
        doc, 
        style="ent", 
        # options=options or COMPETITION_DISPLACY_OPTIONS, 
        jupyter=True
    )

visualize_example(df.iloc[-1])

# print("\n\n... VISUALIZE THE EXAMPLE WITH THE MOST UNIQUE PII ...\n")
# visualize_example(data_df.sort_values(by=["unique_pii_count", "total_pii_count"], ascending=False).reset_index(drop=True).iloc[0])

# print("\n\n... VISUALIZE THE EXAMPLE WITH THE MOST PII TOTAL ...\n")
# visualize_example(data_df.sort_values(by=["total_pii_count", "unique_pii_count"], ascending=False).reset_index(drop=True).iloc[0])

In [ ]:
pii_data = {
    'EMAIL':{},
    'ID_NUM':{},
    'NAME_STUDENT':{},
    'PHONE_NUM':{},
    'STREET_ADDRESS':{},
    'URL_PERSONAL':{},
    'USERNAME':{},
}

def update_pii_data(pii_type, token, tokens, i):
    if pii_type not in pii_data:
        pii_data[pii_type] = {}
    
    # Extracting the token and surrounding context
    token_text = tokens[i]
    surrounding_tokens = tokens[max(0, i-2):i] + tokens[i+1:min(len(tokens), i+3)]
    sentence = ' '.join(tokens)  # Simplified; consider a more accurate sentence detection
    
    # Assuming `tokens` is a list of all tokens in the document and `i` is the index of the current token
    sentence_boundaries = [j for j, token in enumerate(tokens) if token in '.!?'] + [len(tokens)-1]
    sentence_start = max([boundary for boundary in sentence_boundaries if boundary < i]+[0])
    sentence_end = min([boundary for boundary in sentence_boundaries if boundary > i]+[len(tokens)-1])
    sentence_context = tokens[sentence_start:sentence_end+1]
    
    # Determine PII position in the sentence
    position_in_sentence = "middle"
    if i == sentence_start:
        position_in_sentence = "beginning"
    elif i == sentence_end:
        position_in_sentence = "end"
    
    # PII Token Type
    if token_text.isalpha():
        token_type = "alphabetic"
    elif token_text.isdigit():
        token_type = "numeric"
    else:
        token_type = "alphanumeric" if any(char.isalpha() for char in token_text) else "other"
    
    # PII Format Pattern
    format_pattern = ''.join(['d' if char.isdigit() else 'l' if char.isalpha() else char for char in token_text])
    
    # Capitalization
    capitalization = "lowercase"
    if token_text.isupper():
        capitalization = "uppercase"
    elif token_text.istitle():
        capitalization = "titlecase"
    
    # Special Characters
    special_chars = any(not char.isalnum() for char in token_text)
    
    details = {
        'token_text': token_text,
        'surrounding_words': surrounding_tokens,
        'location_in_essay': i,
        'sentence_context': ' '.join(sentence_context),
        'pii_length': len(token_text),
        'position_in_sentence': position_in_sentence,
        'token_type': token_type,
        'format_pattern': format_pattern,
        'capitalization': capitalization,
        'special_chars': special_chars,
        # Add more details as needed
    }
    
    pii_token_key = f"{token_text}_{i}"  # Unique key for each PII instance
    if pii_token_key not in pii_data[pii_type]:
        pii_data[pii_type][pii_token_key] = []
    pii_data[pii_type][pii_token_key].append(details)

# Example of how to call this function within your iteration over the DataFrame
for index, row in tqdm(train_df.iterrows(), total=len(train_df)):
    tokens = row['tokens']
    labels = row['labels']
    for i, label in enumerate(labels):
        if label != 'O':  # If the label indicates PII
            pii_type = label[2:]  # Extract PII type (removing the B- or I- prefix)
            update_pii_data(pii_type, tokens[i], tokens, i)
            
            
# Initialize an empty list to hold each PII instance as a dictionary
flat_pii_data = []

# Iterate through the pii_data dictionary
for pii_type, pii_instances in pii_data.items():
    for instance_key, details_list in pii_instances.items():
        for details in details_list:
            # Create a flat dictionary for each PII instance
            flat_instance = details.copy()  # Start with the existing details
            flat_instance['pii_type'] = pii_type  # Add the PII type
            flat_instance['instance_key'] = instance_key  # Add the instance key for reference
            
            # Append this flat dictionary to our list
            flat_pii_data.append(flat_instance)

# Store and viz
train_pii_df = pd.DataFrame(flat_pii_data)
print("\n... PII DATAFRAME ...\n")
display(train_pii_df)

# Count the number of instances for each PII type
pii_type_counts = train_pii_df['pii_type'].value_counts()
print("\n... PII TYPE COUNTS ...\n")
display(pii_type_counts.to_frame().T)

# Or aggregate to find the average PII length by type
average_pii_length_by_type = train_pii_df.groupby('pii_type')['pii_length'].mean()
print("\n\n... AVERAGE PII LENGTH (CHARS) BY TYPE ...\n")
display(average_pii_length_by_type.to_frame().T)